In [10]:
#
#
# ReadInSARTimeSeries.py : read INSAR time series of cumulation displacement
#                resuling from SBAS Mintpy 
#
# P.Santitamnon (phisan.chula@gmail.com)
# History : v.0.1 , 25 Apr 2022 Initial
#
#
import rasterio as rio 
import pandas as pd
import numpy as np
import h5py
from pathlib import Path
from rasterio.transform import Affine
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import datetime

from fastapi import FastAPI

app = FastAPI()


#############################################################################
class INSAR_TimeSeries:
    def __init__(self, DIR_VELOC ):
        self.DIR_VELOC = DIR_VELOC
        #self.FILE_VELOC   = DIR_VELO.joinpath( 'velocity.h5' )
        #with rio.open( self.FILE_VELOC) as src:
        #    self.VELOC = src.read()
        self.FILE_TIMESR = DIR_VELO.joinpath( 'timeseries.h5' )
        self.ReadH5PY( self.FILE_TIMESR )
        assert( self.ARRAY.shape[0] == len(self.TIMESR) )

    def ReadH5PY( self, FILE_HDF5 ):
        with h5py.File( FILE_HDF5 ) as f:
            #print( f.keys() ); print( f.attrs.keys() )
            self.LENGTH     = f.attrs['LENGTH']
            self.WIDTH      = f.attrs['WIDTH']
            self.EPSG       = f.attrs['EPSG']
            self.UTM_ZONE   = f.attrs['UTM_ZONE']
            self.ORBIT_DIR  = f.attrs['ORBIT_DIRECTION']
            self.START_DATE = f.attrs['START_DATE']
            self.END_DATE   = f.attrs['END_DATE']
            self.NUM_IFG    = int( f.attrs['mintpy.networkInversion.numIfgram'] )
            ################################# 
            self.X_FIRST = float( f.attrs['X_FIRST'] )
            self.X_STEP  = float( f.attrs['X_STEP']  )
            self.Y_FIRST = float( f.attrs['Y_FIRST'] )
            self.Y_STEP  = float( f.attrs['Y_STEP']  )
            #################################
            self.TR = Affine( self.X_STEP, 0.0         , self.X_FIRST, 
                                      0.0, self.Y_STEP , self.Y_FIRST  )
            timesr = pd.DataFrame( list(f['date']), columns=['Date'] ) 
            timesr['Date'] = pd.to_datetime( timesr['Date'].str.decode('utf-8') )
            timesr['DayCnt'] = timesr['Date']-timesr.iloc[0]['Date']
            timesr['DayCnt'] = timesr.DayCnt.dt.days
            self.TIMESR = timesr
            self.ARRAY = np.stack( f['timeseries'] )
   
    def Tr_XY(self, row_col ):
        return self.TR*row_col

    def Tr_RowCol(self, xy ):
        return ~self.TR*xy

    def getTimeSeriesPnt( self, xy ):
        r,c = self.Tr_RowCol( xy ) 
        df = self.TIMESR.copy()
        df['CumDispl_m'] = self.ARRAY[ : , int(c), int(r) ]  # cm
        coeff = np.polyfit( df.DayCnt, df.CumDispl_m, 1 )  # linear regress
        #import pdb; pdb.set_trace()
        def EstDisp( row, coeff ):
            return row.DayCnt*coeff[0] + coeff[1]
        df['CumDispl_m_'] = df.apply( EstDisp, axis='columns', args=[coeff] ) 
        velo_cmy = 100*coeff[0]*365  # average cm per year
        return df,velo_cmy
            
    def PlotTimeSeries( self, dfTIMESR, TITLE, PLOT_FILE=None ):
        fig,ax = plt.subplots()
        dfTIMESR.plot.scatter( "Date", "CumDispl_m", alpha=0.5, 
                               color='r', rot=45, ax=ax )
        dfTIMESR.plot.line( "Date", "CumDispl_m_", alpha=0.5, 
                            lw=5,   color='b', rot=45, ax=ax )
        ax.set_xlabel( '' )
        ax.set_ylabel( 'Cumulation Displacement (m)')
        ax.set_title( TITLE )
        plt.tight_layout()
        if PLOT_FILE is None:
            plt.show()
        else:
            plt.savefig( PLOT_FILE )


#####################################################
#Band 1: -0.0340674
pnt = { 'PLUTALUANG' :  ( 711_666, 1_405_782 ),  }   #  -0.0340674  mm/year
DIR_VELO = Path('')

insar = INSAR_TimeSeries( DIR_VELO )
list_x=[]
list_y=[]
for k,v in pnt.items():
    ts,velo_cmy = insar.getTimeSeriesPnt( v  )
    list_ts = ts.values.tolist()
    for i in list_ts:
      list_x.append(i[0].strftime("%Y-%m-%d"))
      list_y.append(i[2])
    # title = f'{DIR_VELO} "{k}" Avg.Velo.:{velo_cmy:+.2f} cm/year'
    # PLOT_FILE = f'{k}.png' 
    # print( f'Plotting {PLOT_FILE}...' )
    # insar.PlotTimeSeries( ts , title, PLOT_FILE )
json_result = {'CumDispl_m_':list_y,'date':list_x}
print(json_result)
#import pdb; pdb.set_trace()


{'CumDispl_m_': [0.02339158020913601, 0.027825569733977318, 0.041099146008491516, 0.024801917374134064, 0.025752080604434013, 0.010006677359342575, 0.019560078158974648, 0.044774867594242096, 0.003788817673921585, 0.03097836673259735, 0.030921481549739838, 0.02794240228831768, 0.037632208317518234, 0.00694086030125618, 0.024740681052207947, 0.028245974332094193, 0.019889742136001587, 0.017897969111800194, 0.007791211828589439, 0.013332807458937168, 0.015605472959578037, 0.05809078365564346, -0.005120754241943359, 0.009574079886078835, 0.021247921511530876, -0.008811140432953835, 0.0055072009563446045, 0.026552919298410416, 0.037616048008203506, 0.019866663962602615, 0.03770054504275322, 0.013960474170744419, 0.01147509180009365, 0.02116469480097294, 0.0019789710640907288, -0.004157111048698425, -0.0005478952080011368, 0.022021427750587463, 0.007346920669078827, 0.011494316160678864, 0.013668191619217396, 0.01204514317214489, 0.0061566028743982315, 0.010879176668822765, 0.02156012691557